In [0]:
import nltk
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
nltk.download('punkt')
import codecs

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
from google.colab import files
files.upload()

Saving wat.txt to wat.txt


{'wat.txt': b'"Well, Prince, so Genoa and Lucca are now just family estates of the\r\nBuonapartes. But I warn you, if you don\'t tell me that this means war,\r\nif you still try to defend the infamies and horrors perpetrated by that\r\nAntichrist--I really believe he is Antichrist--I will have nothing more\r\nto do with you and you are no longer my friend, no longer my \'faithful\r\nslave,\' as you call yourself! But how do you do? I see I have frightened\r\nyou--sit down and tell me all the news."\r\n\r\nIt was in July, 1805, and the speaker was the well-known Anna Pavlovna\r\nScherer, maid of honor and favorite of the Empress Marya Fedorovna. With\r\nthese words she greeted Prince Vasili Kuragin, a man of high rank and\r\nimportance, who was the first to arrive at her reception. Anna Pavlovna\r\nhad had a cough for some days. She was, as she said, suffering from la\r\ngrippe; grippe being then a new word in St. Petersburg, used only by the\r\nelite.\r\n\r\nAll her invitations without

In [0]:
from nltk.tokenize import word_tokenize

In [0]:
def load_file():
  file = codecs.open("wat.txt",'r',encoding='utf-8',errors='ignore')
  text = file.read()
  file.close()
  return text

In [0]:
import string

In [0]:
def clean_text():
  text = load_file()
  clean_words= []
  text = text.replace("-"," ")
  text = text.replace("--"," ")
  text = text.replace("."," ")
  text  = text.replace("/","")
  words = word_tokenize(text)
  for word in words:
    word = word.strip(string.punctuation)
    if len(word)>=1:
      word = word.lower()
      clean_words.append(word)
  return clean_words

In [0]:

def clean_text():
    text = load_file()
    clean_words=[]
    doc=text.replace("'"," ")
    doc=doc.replace("."," ")
    doc=doc.replace("\n"," ")
    doc=doc.replace("<unk>"," ")
    words=word_tokenize(doc)
    for word in words:
        word=word.strip(string.punctuation)
        if len(word)>=2 and word.isalpha()==True:
            clean_words.append(word.lower())
        else:
            continue
    return clean_words    

In [0]:
def create_dataset():
  words = clean_text()
  dataset = []
  for i in range(11,len(words)):
    seq = words[i-11:i]
    line = ' '.join(seq)
    dataset.append(line)
  return dataset

In [0]:
lines = create_dataset()
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
vocab_size = len(tokenizer.word_index) + 1

In [0]:
import numpy as np
for seq in sequences:
  if len(seq)!=11:
    print(seq)

In [0]:
np_arr = np.array(sequences)
      


In [0]:
np_arr.shape

(102711, 11)

In [0]:
X,Y = np_arr[:,:-1],np_arr[:,-1]
Y = to_categorical(Y,num_classes=vocab_size)

In [0]:
Y.shape

(102711, 7864)

In [0]:
model = Sequential()
model.add(Embedding(vocab_size,10,input_length=X.shape[1]))
model.add(LSTM(1024,return_sequences=True))
model.add(LSTM(1024))
model.add(Dense(1024,activation='relu'))
model.add(Dense(vocab_size,activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 10, 10)            78640     
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 1024)          4239360   
_________________________________________________________________
lstm_2 (LSTM)                (None, 1024)              8392704   
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_2 (Dense)              (None, 7864)              8060600   
Total params: 21,820,904
Trainable params: 21,820,904
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, Y, batch_size=1024, epochs=30)

Instructions for updating:
Use tf.cast instead.
Epoch 1/30
102711/102711 [==============================] - 38s 371us/step - loss: 6.9157 - acc: 0.0537
Epoch 2/30
102711/102711 [==============================] - 34s 327us/step - loss: 6.5993 - acc: 0.0616
Epoch 3/30
102711/102711 [==============================] - 35s 336us/step - loss: 6.5852 - acc: 0.0616
Epoch 4/30
102711/102711 [==============================] - 34s 334us/step - loss: 6.5697 - acc: 0.0616
Epoch 5/30
102711/102711 [==============================] - 34s 333us/step - loss: 6.5469 - acc: 0.0616
Epoch 6/30
102711/102711 [==============================] - 34s 335us/step - loss: 6.3961 - acc: 0.0646
Epoch 7/30
102711/102711 [==============================] - 34s 333us/step - loss: 6.1864 - acc: 0.0783
Epoch 8/30
102711/102711 [==============================] - 35s 337us/step - loss: 6.0217 - acc: 0.0855
Epoch 9/30
102711/102711 [==============================] - 34s 336us/step - loss: 5.8509 - acc: 0.0946
Epoch 10/30
1027

In [0]:
from random import randint

In [0]:
seed_text = X[randint(0,len(X))]

In [0]:
seed_text

array([  66, 1003,   75,    5,  318,   19, 1223,    9,   78,   13])

In [0]:
seed_text = seed_text.reshape(1,10)

In [0]:
model.predict_classes(seed_text)

array([1])

In [0]:
seed_text = lines[randint(0,len(lines))]
seed_text

'table percy drew out the empty chair beside him staring pointedly'

In [0]:
def generate_seq(model,tokenizer,seq_length,seed_text,n_words):
  result = []
  in_text =seed_text
  for i in range(n_words):
    encoded = tokenizer.texts_to_sequences([in_text])[0]
    encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
    yhat = model.predict_classes(encoded)
    out_word = ''
    for word,index in tokenizer.word_index.items():
      if index==yhat:
        out_word = word
        break
    in_text+=' ' + out_word
    result.append(word)
  return ' '.join(result)

In [0]:
from keras.preprocessing.sequence import pad_sequences

In [0]:
generate_seq(model,tokenizer,10,seed_text,1000)

TypeError: ignored

In [0]:
seed_text_new = "dumbledore stood up he stared down at barty crouch for"

In [0]:
generate_seq(model,tokenizer,10,seed_text_new,10000)

"some of the morning of beauxbatons you been snape took the kitchen their hundred figures with his hair s feet with snakes he put to his wand did not find him in azkaban he was wearing a little line of his nose than the usual babble of a bottle of crates lessons many of them were nursing small injuries harry had such a bad case of twitchy ears he had to hold his hands clamped over them as he walked away from the class well rita s definitely not using an invisibility cloak hermione panted five minutes later catching up with harry and ron in the entrance hall and pulling harrys hand away from one of his wiggling ears so that he could hear her moody says he did n't see her anywhere near the judges table at the second task or anywhere near the lake hermione is there any point in telling you to drop this said ron no said hermione stubbornly i want to know how she heard me talking to viktor and how she found out about hagrids mum maybe she had you bugged said harry bugged said ron blankly wh